In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 1, 41), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [3]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=100, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=100, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi
